In [3]:
%%capture
!pip install llama-index==0.10.37 cohere==5.5.0 openai==1.30.1 llama-index-embeddings-openai==0.1.9 llama-index-llms-cohere==0.2.0 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8

In [4]:
import os
import nest_asyncio
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(r"C:\Users\anteb\PycharmProjects\JupyterProject\.env")
nest_asyncio.apply()

# Debug: Print environment variables to verify they are loaded
print("CO_API_KEY:", os.environ.get('CO_API_KEY'))
print("OPENAI_API_KEY:", os.environ.get('OPENAI_API_KEY'))
print("QDRANT_URL:", os.environ.get('QDRANT_URL'))
print("QDRANT_API_KEY:", os.environ.get('QDRANT_API_KEY'))

CO_API_KEY = os.environ.get('CO_API_KEY') or getpass("Enter CO_API_KEY: ")
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or getpass("Enter OPENAI_API_KEY: ")
QDRANT_URL = os.environ.get('QDRANT_URL') or getpass("Enter QDRANT_URL: ")
QDRANT_API_KEY = os.environ.get('QDRANT_API_KEY') or getpass("Enter QDRANT_API_KEY: ")


In [5]:
import requests
from pathlib import Path

# Create directory if it doesn't exist
def create_directory(directory_name):
    path = Path(directory_name)
    path.mkdir(parents=True, exist_ok=True)
    print(f"Directory '{directory_name}' created successfully.")

create_directory("rag_articles")

Directory 'rag_articles' created successfully.


In [6]:
def download_pdf(url, directory):
    # Extract a simple filename from the URL (e.g., "2502.20364.pdf")
    filename = url.split("/")[-1] + ".pdf"
    pdf_path = Path(directory) / filename
    response = requests.get(url)
    response.raise_for_status()  # Optional: raises an exception for bad responses
    with open(pdf_path, "wb") as file:
        file.write(response.content)
    print(f"PDF downloaded and saved to {pdf_path}")
    return str(pdf_path)

In [7]:
# List of PDF links
pdf_links = [
    "https://arxiv.org/pdf/2502.20964",
    "https://arxiv.org/pdf/2502.20969",
    "https://arxiv.org/pdf/2502.20995",
    "https://arxiv.org/pdf/2502.21087",
    "https://arxiv.org/pdf/2502.21263"
]

In [8]:
downloaded_files = []
for url in pdf_links:
    file_path = download_pdf(url, "rag_articles")
    downloaded_files.append(file_path)

PDF downloaded and saved to rag_articles\2502.20964.pdf
PDF downloaded and saved to rag_articles\2502.20969.pdf
PDF downloaded and saved to rag_articles\2502.20995.pdf
PDF downloaded and saved to rag_articles\2502.21087.pdf
PDF downloaded and saved to rag_articles\2502.21263.pdf


In [9]:
# Load the downloaded PDFs using SimpleDirectoryReader from llamaindex
from llama_index.core import SimpleDirectoryReader

# Option 1: Load using a list of file paths
documents = SimpleDirectoryReader(input_files=downloaded_files, filename_as_id=True).load_data()


print(f"Loaded {len(documents)} documents.")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anteb\anaconda3\Lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


Loaded 82 documents.


In [10]:
# Create Node parser
from llama_index.core.node_parser import SentenceSplitter

sentence_splitter = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=16,
    paragraph_separator="\n\n\n\n"
)

In [11]:
# Instantiate embedding model
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [12]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# initialize qdrant client
client = qdrant_client.QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="rag_articles",
    embed_model=embed_model,
)

In [13]:
from llama_index.core import StorageContext

# assign qdrant vector store to storage context
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

In [14]:
from llama_index.core import  VectorStoreIndex
# create the index
index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True,
    store_nodes_override=True,
    transformation=[sentence_splitter],
    embed_model=embed_model,
    storage_context=storage_context,
)

Parsing nodes:   0%|          | 0/82 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/135 [00:00<?, ?it/s]

In [15]:
retirever = index.as_retriever(
    similarity_top_k=5,
    similarity_threshold=0.75)

In [16]:
import qdrant_client
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext

embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

# initialize qdrant client
client = qdrant_client.QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="rag_articles",
    embed_model=embed_model,
)

# assign qdrant vector store to storage context
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    )

# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
    storage_context=storage_context
)

In [17]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.llms.cohere import Cohere

llm = Cohere(model="command-r-plus")

# configure a retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure a post processor
similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.42)

# configure a response sythesizer
response_synthsizer = get_response_synthesizer(llm=llm)

# create a query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)

In [18]:
query_engine.query("How I can improve my AI agent?")

Response(response='You can improve your AI agent by enabling it to take three different actions: Search, Query, and Finish. These actions focus on different aspects of semi-structured data. \n\nThe Search [node] action, for example, focuses on relational information from nodes. This action can be performed on multiple nodes by repeating the same process. It involves finding the most relevant top K neighbours by retrieving all neighbours connected to a given node and then selecting the top K nodes that are most related based on their edges to the node.', source_nodes=[NodeWithScore(node=TextNode(id_='93d30c01-a062-4635-9412-627fecec1549', embedding=None, metadata={'page_label': '5', 'file_name': '2502.21087.pdf', 'file_path': 'rag_articles\\2502.21087.pdf', 'file_type': 'application/pdf', 'file_size': 1340539, 'creation_date': '2025-03-03', 'last_modified_date': '2025-03-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'la